### 1. IMPORTS

In [1]:
from pathlib import Path
import sys
import pandas as pd

### 2. ORGANIZANDO ARQUIVOS

In [2]:
BASE_DIR = Path.cwd().parent
sys.path.append(str(BASE_DIR / "src" / "data_generation"))

from compute_financials import *

# Paths dos arquivos
DATA_RAW = BASE_DIR / "data/raw"
DATA_PROCESSED = BASE_DIR / "data/processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

producao_path = DATA_RAW / "producao_mensal.xlsx"
custos_gerais_path = DATA_RAW / "custos_gerais.xlsx"
processed_path = DATA_PROCESSED / "financials_consolidated.xlsx"

### 3. RODANDO SCRIPTS

In [3]:
# 1. Carregar arquivos
df_prod = pd.read_excel(producao_path)
df_custos_gerais = pd.read_excel(custos_gerais_path)

In [4]:
# 2. Aplicar share de custos gerais
df_prod = aplicar_share_custos_gerais(df_prod, df_custos_gerais)

In [5]:
# 3. Calcular lucro líquido
df_prod = calcular_lucro_liquido(df_prod)

In [6]:
# 4. Consolidar agregados mensais
df_agg = consolidar_agregados(df_prod)

### 4. SALVAR ARQUIVOS

In [7]:
salvar_financeiro(df_prod, df_agg, producao_path, processed_path)

✅ Produção mensal atualizada salva em: c:\Users\usuario\Documents\bagunca do pedro\Projetos Ciencia de Dados\Projeto Empresa Ficticia\Pretoleira Gamarra\data\raw\producao_mensal.xlsx
✅ Agregados mensais salvos em: c:\Users\usuario\Documents\bagunca do pedro\Projetos Ciencia de Dados\Projeto Empresa Ficticia\Pretoleira Gamarra\data\processed\financials_consolidated.xlsx


### 5. INSPEÇÃO

In [8]:
print("Amostra de produção mensal com lucro líquido:")
display(df_prod.head())

print("Amostra de agregados mensais:")
display(df_agg.head())

Amostra de produção mensal com lucro líquido:


,campo_id,nome_campo,estado,tipo_petroleo,data,producao_barris,preco_brl,receita,custo_operacional,lucro_bruto,...,custo_fixo_brl,margem_bruta,admin_brl,manutencao_brl,logistica_brl,share_admin,share_manut,share_logistica,custo_geral_brl,lucro_liquido_brl
0,1,Campo Voluptatem Salmão,ES,médio,2005-01-01,851115.946778,156.5130,1.332107e+08,4.256080e+07,7.992643e+07,...,5000,9.064991e+07,50000.000000,40000.000000,30000.000000,26450.739662,21160.591730,15870.443797,63481.775190,9.058643e+07
1,1,Campo Voluptatem Salmão,ES,médio,2005-02-01,811212.857933,158.0032,1.281742e+08,4.056564e+07,7.690454e+07,...,5000,8.760858e+07,51250.000000,41000.000000,30750.000000,26333.678996,21066.943197,15800.207398,63200.829592,8.754538e+07
2,1,Campo Voluptatem Salmão,ES,médio,2005-03-01,959857.581499,161.1884,1.547179e+08,4.799788e+07,9.283074e+07,...,5000,1.067200e+08,52165.063509,41732.050808,31299.038106,26984.903185,21587.922548,16190.941911,64763.767644,1.066553e+08
3,1,Campo Voluptatem Salmão,ES,médio,2005-04-01,893708.157978,163.4040,1.460355e+08,4.469041e+07,8.762129e+07,...,5000,1.013451e+08,52500.000000,42000.000000,31500.000000,25456.385124,20365.108099,15273.831075,61095.324298,1.012840e+08
4,1,Campo Voluptatem Salmão,ES,médio,2005-05-01,949636.366959,164.2230,1.559521e+08,4.748682e+07,9.357128e+07,...,5000,1.084653e+08,52165.063509,41732.050808,31299.038106,26058.811422,20847.049137,15635.286853,62541.147412,1.084028e+08


Amostra de agregados mensais:


,data,producao_total_barris,receita_total_brl,custo_operacional_total_brl,custo_geral_total_brl,lucro_total_brl
0,2005-01-01,1.608870e+06,2.518090e+08,8.045349e+07,120000.000000,1.712356e+08
1,2005-02-01,1.578764e+06,2.494497e+08,7.894819e+07,123000.000000,1.703785e+08
2,2005-03-01,1.855520e+06,2.990883e+08,9.278601e+07,125196.152423,2.061771e+08
3,2005-04-01,1.843140e+06,3.011764e+08,9.216699e+07,126000.000000,2.088834e+08
4,2005-05-01,1.901002e+06,3.121882e+08,9.506008e+07,125196.152423,2.170029e+08


### 6. CHECAGENS RAPIDAS

In [10]:
# Margem média mensal
df_prod["margem_liquida_pct"] = df_prod["lucro_liquido_brl"] / df_prod["receita"]
margem_media = df_prod["margem_liquida_pct"].mean() * 100
print(f"\nMargem líquida média: {margem_media:.2f}%")
# Verificar se o lucro líquido total é positivo
lucro_total = df_prod["lucro_liquido_brl"].sum()
if lucro_total > 0:
    print(f"Lucro líquido total positivo: R${lucro_total:,.2f}")
else:
    print(f"Lucro líquido total negativo: R${lucro_total:,.2f}")


Margem líquida média: 64.28%
Lucro líquido total positivo: R$247,486,564,435.45
